# Getting up to speed with Dask

## Part 2: Dask to the rescue!

We will do the same analysis as Part 1 with the same laptop-sized machine, except we will be using a rougly 10x larger dataset

AWS EC2 instance type: r5.xlarge (2 CPU, 16GB RAM)

Dask will work without intializing a distributed backend, but it's [recommended to do so](https://docs.dask.org/en/latest/setup/single-distributed.html) even on a laptop. To initialize, start a `Client` like so:

In [1]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:39277 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 64 Memory: 270.47 GB


In [2]:
import pandas as pd
import numpy as np
import datetime
import s3fs
from pathlib import Path
import os

# change this path if you changed in Part 0
#data_path = Path('data')
data_path = '/home/ubuntu/getting-up-to-speed-with-dask/data'
seed = 42

# Load and explore data

In [3]:
taxi_dtypes = {
    'store_and_fwd_flag': str,
    'RatecodeID': 'float64',
    'VendorID': 'float64',
    'passenger_count': 'float64',
    'payment_type': 'float64',
}

def load_csv(f):
    """ Same as Part 1"""
    return pd.read_csv(
        f,
        dtype=taxi_dtypes, 
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [4]:
# new dask imports
from dask import delayed
import dask.dataframe as dd

## The hard way

We could parallelize our for-loop that blew up in Part 1 using [dask.delayed](https://docs.dask.org/en/latest/delayed.html).

NOTE: There is a better way below! This is just for educational purposes.

In [5]:
%%time

dfs = []
#for f in data_path.iterdir():
count=0
for f in os.listdir(data_path):
    print(data_path+'/'+f)   
    df = delayed(load_csv)(data_path+'/'+f)  # see the delayed() call
    dfs.append(df)
    count+=1
    if count >= 2:
        print(count)
        break

/home/ubuntu/getting-up-to-speed-with-dask/data/yellow_tripdata_2019-02.csv
/home/ubuntu/getting-up-to-speed-with-dask/data/yellow_tripdata_2019-03.csv
2
CPU times: user 2.22 ms, sys: 966 µs, total: 3.18 ms
Wall time: 2.36 ms


In [6]:
taxi = dd.from_delayed(dfs)  # from_delayed rather than pd.concat

In [7]:
dfs

[Delayed('load_csv-95cc1624-8767-4dd0-b660-a056e5d09ff5'),
 Delayed('load_csv-18fe021f-c010-4552-9523-0112630ade9e')]

In [34]:
%%time
df0 = dfs[0].compute()

CPU times: user 1.24 s, sys: 4.24 s, total: 5.48 s
Wall time: 19.9 s


In [35]:
df0.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-02-01 00:59:04,2019-02-01 01:07:27,1.0,2.1,1.0,N,48,234,1.0,9.0,0.5,0.5,2.0,0.0,0.3,12.3,0.0
1,1.0,2019-02-01 00:33:09,2019-02-01 01:03:58,1.0,9.8,1.0,N,230,93,2.0,32.0,0.5,0.5,0.0,0.0,0.3,33.3,0.0
2,1.0,2019-02-01 00:09:03,2019-02-01 00:09:16,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
3,1.0,2019-02-01 00:45:38,2019-02-01 00:51:10,1.0,0.8,1.0,N,95,95,2.0,5.5,0.5,0.5,0.0,0.0,0.3,6.8,0.0
4,1.0,2019-02-01 00:25:30,2019-02-01 00:28:14,1.0,0.8,1.0,N,140,263,2.0,5.0,0.5,0.5,0.0,0.0,0.3,6.3,0.0


In [36]:
type(df0)

pandas.core.frame.DataFrame

In [37]:
taxi

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_year_seconds,pickup_week_hour
npartitions=127,,,,,,,,,,,,,,,,,,,,,,,,
,float64,datetime64[ns],datetime64[ns],float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
%%time
len(taxi)

CPU times: user 423 ms, sys: 415 ms, total: 839 ms
Wall time: 16 s


14851920

## Okay, let's do it the better way

`dask.dataframe.read_csv` has the same API as `pandas.dataframe.read_csv`

In [39]:
%%time

taxi = dd.read_csv(
    'data/yellow_tripdata_2019-*.csv',
    dtype=taxi_dtypes, 
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

CPU times: user 32.1 ms, sys: 0 ns, total: 32.1 ms
Wall time: 29.8 ms


In [40]:
%%time
len(taxi)

CPU times: user 2.46 s, sys: 271 ms, total: 2.73 s
Wall time: 16.9 s


84399019

<br>

Note the calls to `.compute()`. Dask executes _lazily_ meaning it won't do something unless you explicitly tell it to. Peek at the dashboard while these are executing to see the progress!

In [15]:
%%time
taxi.memory_usage(deep=True).sum().compute() / 1e9

CPU times: user 4.95 s, sys: 624 ms, total: 5.58 s
Wall time: 2min 26s


16.367014316

<br>
This takes a while

In [16]:
%%time
np.round(taxi.describe().compute(), 3).T

CPU times: user 3min 9s, sys: 10.4 s, total: 3min 19s
Wall time: 17min 55s


,count,mean,std,min,25%,50%,75%,max
VendorID,84152418.0,1.645,0.498,1.00,1.00,2.00,2.00,4.00
passenger_count,84152418.0,1.563,1.208,0.00,1.00,1.00,2.00,9.00
trip_distance,84399019.0,3.001,8.091,-37264.53,1.07,1.93,8.82,45977.22
RatecodeID,84152418.0,1.061,0.760,1.00,1.00,1.00,1.00,99.00
PULocationID,84399019.0,163.158,66.016,1.00,132.00,162.00,234.00,265.00
DOLocationID,84399019.0,161.353,70.251,1.00,116.00,163.00,236.00,265.00
payment_type,84152418.0,1.289,0.479,1.00,1.00,1.00,2.00,5.00
fare_amount,84399019.0,13.344,174.375,-1856.00,7.00,11.00,32.04,943274.80
extra,84399019.0,1.087,1.249,-60.00,0.00,1.00,3.00,535.38
mta_tax,84399019.0,0.495,0.067,-0.50,0.50,0.50,0.50,212.42


# Feature engineering

In [9]:
def make_features(df):
    """ Same code from Part 1 """
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    #df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.isocalendar().week.astype(int)
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df['pickup_year_seconds'] = (df.tpep_pickup_datetime - datetime.datetime(2019, 1, 1, 0, 0, 0)).dt.seconds
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['store_and_fwd_flag'] = (df.store_and_fwd_flag == 'Y').astype(int)
    df['VendorID'] = df.VendorID.fillna(-1)
    df['RatecodeID'] = df.RatecodeID.fillna(-1)

In [42]:
%%time

make_features(taxi)

CPU times: user 80.7 ms, sys: 0 ns, total: 80.7 ms
Wall time: 76.5 ms


In [43]:
%%time

taxi.head()

CPU times: user 152 ms, sys: 28.2 ms, total: 180 ms
Wall time: 2.18 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_year_seconds,pickup_week_hour
0,1.0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,0,151,239,1.0,...,0.0,0.3,9.95,NaN,1,1,0,46,2800,24
1,1.0,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,0,239,246,1.0,...,0.0,0.3,16.30,NaN,1,1,0,59,3587,24
2,2.0,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,0,236,236,1.0,...,0.0,0.3,5.80,NaN,4,51,13,48,49710,109
3,2.0,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,0,193,193,2.0,...,0.0,0.3,7.55,NaN,2,48,15,52,57145,63
4,2.0,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,0,193,193,2.0,...,0.0,0.3,55.55,NaN,2,48,15,56,57417,63


# Machine learning

In [44]:
# same as Part 1
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_minute', 
    'pickup_year_seconds',
    'pickup_week_hour', 
    'passenger_count',
]
categorical_feat = [
    'VendorID', 
    'RatecodeID', 
    'store_and_fwd_flag',
    'PULocationID',
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'total_amount'

### ML with Dask

In [27]:
# note the dask_ml imports rather than sklearn
from dask_ml.model_selection import train_test_split
from dask_ml.metrics import mean_squared_error
from dask_ml.xgboost import XGBRegressor

In [28]:
%%time

X_train, X_test, y_train, y_test = train_test_split(
    taxi[features], taxi[y_col], test_size=0.33, random_state=seed, shuffle=True)

CPU times: user 8.22 ms, sys: 0 ns, total: 8.22 ms
Wall time: 7.17 ms


<br>

Remember, nothing happens yet because of lazy evaluation. If you have the RAM, you can call `df.persist()` to avoid repeated CSV loading

(but if you had the RAM, you could do without Dask)

In [29]:
# X_train.persist()
# y_train.persist()

In [30]:
xgb = XGBRegressor(
    n_estimators=10, 
    max_depth=3, 
    learning_rate=0.1, 
    random_state=seed, 
    n_jobs=-1,
)

In [31]:
%%time

_ = xgb.fit(X_train, y_train)

CPU times: user 16.4 s, sys: 536 ms, total: 17 s
Wall time: 2min


In [32]:
%%time

# get test RMSE
preds = xgb.predict(X_test)
np.sqrt(mean_squared_error(preds, y_test.to_dask_array()))

CPU times: user 6.35 s, sys: 0 ns, total: 6.35 s
Wall time: 25.5 s


145.07297112930445

### ML with SKLearn

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [11]:
# features and target column names
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_minute', 
    'pickup_year_seconds',
    'pickup_week_hour', 
    'passenger_count',
]
categorical_feat = [
    'VendorID', 
    'RatecodeID', 
    'store_and_fwd_flag',
    'PULocationID',
    'DOLocationID',
]
features = numeric_feat #+ categorical_feat
y_col = 'total_amount'

In [12]:
%%time

make_features(taxi)

CPU times: user 79.9 ms, sys: 1.52 ms, total: 81.4 ms
Wall time: 77.3 ms


In [13]:
%%time

X_train, X_test, y_train, y_test = train_test_split(
    taxi[features], taxi[y_col], test_size=0.33, random_state=seed)

NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.